# Assignment - Segmenting and clustering neighborhoods in Toronto

-  Scrape code from the following [Wikipedia page](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)
-  Obtain data that is in the list of postal codes 
-  Transform the data into a pandas dataframe

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood


In [2]:
import pandas as pd
import numpy as np
#Beautiful Soup is a Python library for pulling data out of HTML and XML files.
!pip install BeautifulSoup4
from bs4 import BeautifulSoup
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


print('Libraries imported.')

#  'None' value means unlimited.
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


Requirement not upgraded as not directly required: BeautifulSoup4 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


In [3]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


In [4]:
#columns = ['PostalCode', 'Borough', 'Neighborhood']
#df_Toronto = pd.DataFrame(columns = columns)
#df_Toronto

In [5]:
url_postalcodes = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
!wget -O 'toronto_postal_codes.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

print('Downloaded file')

with open('toronto_postal_codes.html') as postalcodes:
    soup = BeautifulSoup(postalcodes, 'html.parser')

#soup = BeautifulSoup(urlopen(url_postalcodes), "html5lib")

--2018-12-20 17:27:47--  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
Resolving en.wikipedia.org (en.wikipedia.org)... 91.198.174.192, 2620:0:862:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|91.198.174.192|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80280 (78K) [text/html]
Saving to: ‘toronto_postal_codes.html’

100%[======================================>] 80,280       312KB/s   in 0.3s   

2018-12-20 17:27:47 (312 KB/s) - ‘toronto_postal_codes.html’ saved [80280/80280]

Downloaded file


Beautiful Soup - 

Reference code for parsing td and tr tags
[https://www.airpair.com/python/posts/using-python-and-qgis-for-geospatial-visualization#rmxtgga1m5PJ9Vp8.30](Using Python and QGIS for geospatial visualizations - a Case Study)

In [6]:

soup.prettify()
#The “default” manner to create a DataFrame from python is to use a list of dictionaries. 
#In this case each dictionary key is used for the column headings. A default index will be created automatically:

data = []
#print(df_Toronto)
if ('wikitable' in soup.table['class']):
    for row in soup.table.tbody('tr'):
        if row('th'):
            continue  # header row
        row_data = row('td')
        if (row_data[1].text == 'Not assigned'):
            continue
         
        data.append({'PostalCode': row_data[0].text,
                                        'Borough': row_data[1].text,
                                        'Neighborhood': row_data[2].text})
        
df_Toronto = pd.DataFrame(data)
df_Toronto.shape


(212, 3)

In [7]:
df_Toronto.columns

Index(['Borough', 'Neighborhood', 'PostalCode'], dtype='object')

In [8]:
df_Toronto.head(10)

,Borough,Neighborhood,PostalCode
0,North York,Parkwoods\n,M3A
1,North York,Victoria Village\n,M4A
2,Downtown Toronto,Harbourfront\n,M5A
3,Downtown Toronto,Regent Park\n,M5A
4,North York,Lawrence Heights\n,M6A
5,North York,Lawrence Manor\n,M6A
6,Queen's Park,Not assigned\n,M7A
7,Etobicoke,Islington Avenue\n,M9A
8,Scarborough,Rouge\n,M1B
9,Scarborough,Malvern\n,M1B


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

Good [tutorial](https://www.youtube.com/watch?v=Wb2Tp35dZ-I) on pandas groupby - split, apply, combine

Perform a computation on the grouped data
For each group combine the neighborhood column values (Aggregation)

In [9]:
df_Toronto['Neighborhood'] = df_Toronto['Neighborhood'].map(lambda x:x.rstrip("\n"))

In [10]:
df_Toronto.head(10)

,Borough,Neighborhood,PostalCode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,Harbourfront,M5A
3,Downtown Toronto,Regent Park,M5A
4,North York,Lawrence Heights,M6A
5,North York,Lawrence Manor,M6A
6,Queen's Park,Not assigned,M7A
7,Etobicoke,Islington Avenue,M9A
8,Scarborough,Rouge,M1B
9,Scarborough,Malvern,M1B


Groupby is applied on columns 'PostalCode' and 'Borough'. With agg function you can use different functions on different columns. Concatenate the elements in column 'Neighborhood' while inserting a ', ' between the words 

The result of the aggregation will have the group names as the new index along the grouped axis. In the case of multiple keys, the result is a MultiIndex by default, though this can be changed by using the as_index option:

In [11]:
df_Toronto = df_Toronto.groupby(['PostalCode', 'Borough'], as_index = False).agg({'Neighborhood' : ', '.join})

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

Returns a pd.Series object consisting of the 'Neighborhood' column for all rows where 'Neighborhood' is 'Not assigned'
We assign df_Toronto['Borough'] for all elements in column 'Neighborhood' where 'Neighborhood' was 'Not assigned'

In [12]:
df_Toronto.loc[df_Toronto['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df_Toronto['Borough']

In [13]:
df_Toronto.shape

(103, 3)

Part 2 - Get the latitude and the longitude coordinates of each neighborhood. Use the [Geocoder](https://geocoder.readthedocs.io/index.html) Python package.

In [14]:
def get_coordinates(code):
    lat_lng_coords = None
    
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(code))
      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return (latitude, longitude)

In [15]:
!wget -O "postalcode_coords.csv" http://cocl.us/Geospatial_data 


--2018-12-20 17:27:52--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2018-12-20 17:27:52--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2018-12-20 17:27:53--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.25.197
Connecting to ibm.box.com (ibm.box.com)|107.152.25.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2018-12-20 17:27:53--  https://ibm.ent.box.com/shared/

In [16]:
df_coordinates = pd.read_csv("postalcode_coords.csv")

In [17]:
df_coordinates.shape

(103, 3)

In [18]:
df_Toronto.shape

(103, 3)

In [19]:
df_Toronto

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [20]:
df_Toronto = df_Toronto.join(df_coordinates[['Latitude', 'Longitude']])

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word **Toronto**.

Just make sure:

to add enough Markdown cells to explain what you decided to do and to report any observations you make.
to generate maps to visualize your neighborhoods and how they cluster together.
Once you are happy with your analysis, submit a link to the new Notebook on your Github repository.

Use the geopy library to get the latitude and the longitude coordinates of Toronto, ON

In [21]:
from geopy.geocoders import Nominatim

address = "Toronto, ON"

geolocator = Nominatim(user_agent="My - Application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)


43.653963 -79.387207


Create a map of Toronto with Borough's that contain the work Toronto mapped on to it

In [22]:
# Mask
Toronto_boroughs = list(map(lambda x : x.__contains__("Toronto"), df_Toronto['Borough']))

In [23]:
df_TorontoBoroughs = df_Toronto[Toronto_boroughs].reset_index(drop=True)

In [24]:
df_TorontoBoroughs

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


## Create a map of Toronto with the above neigborhoods mapped on to it.

In [25]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add neighborhood borough markers to map
for lat, lng, borough, neighborhood in zip(df_TorontoBoroughs['Latitude'], df_TorontoBoroughs['Longitude'], df_TorontoBoroughs['Borough'], df_TorontoBoroughs['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    popup_label = folium.Popup(label, parse_html=True)
    
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=popup_label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
map_toronto

## Segment and cluster the neighborhoods in Downtown Toronto

In [26]:
df_TorontoBoroughs.shape

(38, 5)

In [27]:
address = "Downtown Toronto, ON"
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("Latitude and Longitude = {}, {}".format(latitude, longitude))

Latitude and Longitude = 43.655115, -79.380219


In [28]:
df_DowntownToronto = df_Toronto[df_Toronto['Borough'] == "Downtown Toronto"].reset_index(drop=True)

In [69]:
print("There are {} neighborhoods in Downtown Toronto".format(df_DowntownToronto.shape[0]))

There are 18 neighborhoods in Downtown Toronto


In [29]:
df_DowntownToronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


Let's visualize Downtown Toronto and the neighborhoods in it.

#### Create a map of Downtown Toronto with latitude and longitude values
Zooming in on Downtown Toronto

In [31]:
# create map of Downtown Toronto using latitude and longitude values
map_downtownToronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add circle markers for each of the neighborhoods to the map
# parse_html - True if the popup is a template that needs to the rendered first
for lat, lng, neighborhood in zip(df_DowntownToronto['Latitude'], df_DowntownToronto['Longitude'], df_DowntownToronto['Neighborhood']):
    label = folium.Popup(neighborhood, parse_html=True)
    
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtownToronto)  
    
map_downtownToronto

## Use the FourSquare API to explore the neighborhoods in Downtown Toronto and segment them

In [124]:
# The code was removed by Watson Studio for sharing.

#### Let's explore the first neighborhood in our dataframe.

In [125]:
neighborhood_name = df_DowntownToronto.loc[0, 'Neighborhood']
neighborhood_latitude = df_DowntownToronto.loc[0, 'Latitude']
neighborhood_longitude = df_DowntownToronto.loc[0, 'Longitude']

print("Latitude and Longitude values of {} are {} and {}".format(neighborhood_name, neighborhood_latitude, 
                                                                 neighborhood_longitude))



Latitude and Longitude values of Rosedale are 43.6795626 and -79.37752940000001


#### Now, let's get the top 100 venues that are in Rosedale  within a radius of 500 meters.

#### First, let's create the GET request URL.

In [126]:

RADIUS = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION,
                                                                                    neighborhood_latitude, neighborhood_longitude, RADIUS, LIMIT)


Send the GET request and examine the resutls

In [127]:
results = requests.get(url).json()
results['response']['groups'][0]['items']

[{'reasons': {'count': 0,
   'items': [{'reasonName': 'globalInteractionReason',
     'summary': 'This spot is popular',
     'type': 'general'}]},
  'referralId': 'e-0-4aff2d47f964a520743522e3-0',
  'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/playground_',
      'suffix': '.png'},
     'id': '4bf58dd8d48988d1e7941735',
     'name': 'Playground',
     'pluralName': 'Playgrounds',
     'primary': True,
     'shortName': 'Playground'}],
   'id': '4aff2d47f964a520743522e3',
   'location': {'address': '38 Scholfield Ave.',
    'cc': 'CA',
    'city': 'Toronto',
    'country': 'Canada',
    'crossStreet': 'at Edgar Ave.',
    'distance': 327,
    'formattedAddress': ['38 Scholfield Ave. (at Edgar Ave.)',
     'Toronto ON',
     'Canada'],
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.68232820227814,
      'lng': -79.37893434347683}],
    'lat': 43.68232820227814,
    'lng': -79.37893434347683,
    'state': 'ON'},
   'name

All the information is in the *items* key. 

In [128]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [129]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

nearby_venues.head()

,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,venue.location.distance,venue.location.formattedAddress,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.state,venue.name,venue.photos.count,venue.photos.groups
0,0,"[{'reasonName': 'globalInteractionReason', 'ty...",e-0-4aff2d47f964a520743522e3-0,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,4aff2d47f964a520743522e3,38 Scholfield Ave.,CA,Toronto,Canada,at Edgar Ave.,327,"[38 Scholfield Ave. (at Edgar Ave.), Toronto O...","[{'lng': -79.37893434347683, 'label': 'display...",43.682328,-79.378934,ON,Rosedale Park,0,[]
1,0,"[{'reasonName': 'globalInteractionReason', 'ty...",e-0-4bd777aa5cf276b054639b00-1,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,4bd777aa5cf276b054639b00,NaN,CA,NaN,Canada,NaN,408,[Canada],"[{'lng': -79.37378835021306, 'label': 'display...",43.682036,-79.373788,NaN,Whitney Park,0,[]
2,0,"[{'reasonName': 'globalInteractionReason', 'ty...",e-0-4d0e77df76cc37045715767c-2,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,4d0e77df76cc37045715767c,107 Crescent Road,CA,Toronto,Canada,South Drive,444,"[107 Crescent Road (South Drive), Toronto ON, ...","[{'lng': -79.38277328698108, 'label': 'display...",43.678300,-79.382773,ON,Alex Murray Parkette,0,[]
3,0,"[{'reasonName': 'globalInteractionReason', 'ty...",e-0-4ef8f2a3775b54cdb5bdec7c-3,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,4ef8f2a3775b54cdb5bdec7c,South Dr,CA,Toronto,Canada,at Glen Rd,464,"[South Dr (at Glen Rd), Toronto ON, Canada]","[{'lng': -79.37384239440172, 'label': 'display...",43.676352,-79.373842,ON,Milkman's Lane,0,[]


In [130]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
# .loc is very useful to slice the dataframe along the axis 0 or 1

nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
# venue.categories checked for each row .. reduce from a json to a string
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
# list comprehension
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Rosedale Park,Playground,43.682328,-79.378934
1,Whitney Park,Park,43.682036,-79.373788
2,Alex Murray Parkette,Park,43.678300,-79.382773
3,Milkman's Lane,Trail,43.676352,-79.373842


In [131]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


2. Explore neighborhoods in Downtown Toronto
#### Let's create a function to repeat the same process to all the neighborhoods in Downtown Toronto

In [132]:
# @hiddencell
# names - neighborhoods e.g. Rosedale
# latitudes 
# longitudes
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()
        
        venues = results['response']['groups'][0]['items']
        
        # Put the information in the list
        
        # return only relevant information for each nearby venue
        # 1 list for each neigborhood
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venues])

    nearby_venues = pd.DataFrame([item for neighborhood_venue_list in venues_list for item in neighborhood_venue_list])
    
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [133]:
#### Run the above function on each neighborhood and create a new dataframe called *downtown_toronto_venues*.

In [134]:
downtown_toronto_venues = getNearbyVenues(df_DowntownToronto['Neighborhood'], df_DowntownToronto['Latitude'], df_DowntownToronto['Longitude'])
downtown_toronto_venues.shape

(1275, 7)

In [135]:
downtown_toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"Cabbagetown, St. James Town",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


#### Count of venues returned by neighborhood

In [136]:
downtown_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"Cabbagetown, St. James Town",50,50,50,50,50,50
Central Bay Street,79,79,79,79,79,79
"Chinatown, Grange Park, Kensington Market",95,95,95,95,95,95
Christie,15,15,15,15,15,15
Church and Wellesley,86,86,86,86,86,86
"Commerce Court, Victoria Hotel",100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues

In [137]:
print('There are {} uniques categories.'.format(len(downtown_toronto_venues['Venue Category'].unique())))

There are 200 uniques categories.


In [138]:
downtown_toronto_venues['Venue Category'].unique()

array(['Playground', 'Park', 'Trail', 'Diner', 'Indian Restaurant',
       'Italian Restaurant', 'Japanese Restaurant', 'Café', 'Restaurant',
       'Butcher', 'General Entertainment', 'Bakery', 'Jewelry Store',
       'Pet Store', 'Gastropub', 'Pub', 'Caribbean Restaurant',
       'Taiwanese Restaurant', 'Gift Shop', 'Deli / Bodega', 'Coffee Shop',
       'Market', 'Thai Restaurant', 'Bank', 'Liquor Store', 'Beer Store',
       'Sandwich Place', 'Pizza Place', 'Breakfast Spot', 'Pharmacy',
       'Ice Cream Shop', 'Chinese Restaurant', 'Grocery Store',
       'Snack Place', 'Flower Shop', 'Dance Studio', 'Bubble Tea Shop',
       'Ramen Restaurant', 'Bookstore', 'Salon / Barbershop', 'Bar',
       'Tea Room', 'Burger Joint', 'Juice Bar', 'Piano Bar',
       'Mexican Restaurant', 'Adult Boutique', 'Creperie',
       'Ethiopian Restaurant', 'Hobby Shop', 'Vietnamese Restaurant',
       'Gay Bar', 'Sushi Restaurant', 'Steakhouse', "Men's Store",
       'Dog Run', 'Sports Bar', 'Smoke Sho

There is a venue category by name 'Neighborhood' which can get confused with our existing column 'Neighborhood'.
Let's rename this venue category before we do the one hot encoding

In [139]:
downtown_toronto_venues.loc[downtown_toronto_venues["Venue Category"] == "Neighborhood", 'Venue Category'] = "Neighborhood Venue"

## 3. Analyze Each Neighborhood

In [140]:
# one hot encoding
downtown_toronto_onehot = pd.get_dummies(downtown_toronto_venues[['Venue Category']], prefix="", prefix_sep="")



In [141]:
# add neighborhood column back to dataframe
downtown_toronto_onehot['Neighborhood'] = downtown_toronto_venues['Neighborhood'] 
downtown_toronto_onehot.head()

,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood Venue,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [142]:
# move neighborhood column to the first column
fixed_columns = [downtown_toronto_onehot.columns[-1]] + list(downtown_toronto_onehot.columns[:-1])

In [143]:
downtown_toronto_onehot = downtown_toronto_onehot[fixed_columns]
downtown_toronto_onehot.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood Venue,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

## Analyze each neighborhood

In [144]:
downtown_toronto_onehot.shape

(1275, 201)

Now, let's group rows by neighborhood and by taking the mean of the occurence of each category

In [145]:
downtown_toronto_grouped = downtown_toronto_onehot.groupby('Neighborhood').mean().reset_index()
downtown_toronto_grouped[0:5]


,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood Venue,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.0,0.0,0.0,0.010000,0.010000,0.0,0.03,0.0,0.0,0.000000,0.030000,0.00,0.030000,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.02,0.000000,0.01,0.020000,0.0,0.000000,0.020000,0.01,0.00,0.050000,0.00,0.000000,0.000000,0.0,0.0,0.030000,0.010000,0.060000,0.0,0.0,0.0,0.000000,0.000000,0.020000,0.0,0.020000,0.000000,0.0,0.01,0.010000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.01,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0.01,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.020000,0.0,0.00,0.01,0.0,0.00,0.01,0.0,0.010000,0.00,0.03,0.010000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.030000,0.0,0.010000,0.010000,0.0,0.0,0.000000,0.020000,0.010000,0.00,0.0,0.000000,0.0,0.01,0.000000,0.010000,0.00,0.0,0.01,0.0,0.0,0.000000,0.000000,0.01,0.0,0.01,0.0,0.000000,0.0,0.01,0.01,0.000000,0.01,0.010000,0.0,0.0,0.000000,0.0,0.0,0.00,0.00,0.0,0.02,0.000000,0.00,0.01,0.000000,0.000000,0.01,0.000000,0.000000,0.01,0.030000,0.0,0.010000,0.01,0.000000,0.0,0.000000,0.010000,0.0,0.010000,0.0,0.01,0.000000,0.00,0.0,0.000000,0.01,0.0,0.0,0.0

Let's confirm the new size

In [146]:
print("Confirming there are {} neighborhoods with {} venues in Downtown Toronto".format(downtown_toronto_grouped.shape[0], downtown_toronto_grouped.shape[1]))

Confirming there are 18 neighborhoods with 201 venues in Downtown Toronto


### Let's print each neighborhood along with the top 5 common venues

In [147]:
num_top_venues = 5


for hood in downtown_toronto_grouped['Neighborhood']:
    print("------"+hood+"------")
    #Transpose the index and columns
    
    temp = downtown_toronto_grouped[downtown_toronto_grouped["Neighborhood"] == hood].T.reset_index()
    #print(temp[0:5])
    #  row 0 (in the transposed form) is the Neighborhood name, we will skip this row name from our calculations
    temp.columns =['venue', 'freq']
    
    # Select second rown onwards (Interested in the mean frequency values)
    # Selecting rows in a dataframe by index ..use iloc
    temp = temp.iloc[1:]
    i = i + 1
    temp['freq'] = temp['freq'].astype(float)
    
    temp = temp.round({'freq' : 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

------Adelaide, King, Richmond------
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3      Thai Restaurant  0.04
4  American Restaurant  0.04


------Berczy Park------
                venue  freq
0         Coffee Shop  0.07
1          Restaurant  0.05
2        Cocktail Bar  0.05
3  Seafood Restaurant  0.04
4      Farmers Market  0.04


------CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara------
              venue  freq
0    Airport Lounge  0.14
1   Airport Service  0.14
2  Airport Terminal  0.14
3             Plane  0.07
4           Airport  0.07


------Cabbagetown, St. James Town------
               venue  freq
0        Coffee Shop  0.10
1         Restaurant  0.08
2               Café  0.06
3        Pizza Place  0.06
4  Indian Restaurant  0.04


------Central Bay Street------
                venue  freq
0         Coffee Shop  0.16
1  Italian Restaurant  0.

### Are Coffee Shops most popular in Downtown Toronto. Let's find out!
Let's put the data above into a pandas dataframe
First let's write a function to sort the venues in descending order of frequency.

In [148]:
# row is a series
def return_most_common_venues(ind, row, num_top_venues):
    row_categories = row.iloc[1:]
    # work with the series and sort it.
    
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    if (ind == 1):
        print(row_categories_sorted.index.values[0:num_top_venues])
        
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood

In [149]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to the number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_toronto_grouped['Neighborhood']

for ind in np.arange(downtown_toronto_grouped.shape[0]):
    # Set the values of the most common venue categories
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ind, downtown_toronto_grouped.iloc[ind, :], num_top_venues)
                       
neighborhoods_venues_sorted                      

['Coffee Shop' 'Restaurant' 'Cocktail Bar' 'Seafood Restaurant'
 'Farmers Market' 'Bakery' 'Italian Restaurant' 'Steakhouse' 'Cheese Shop'
 'Café']


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Asian Restaurant,Hotel,Bakery,Bar,Clothing Store
1,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Seafood Restaurant,Farmers Market,Bakery,Italian Restaurant,Steakhouse,Cheese Shop,Café
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Boutique,Airport,Airport Food Court,Airport Gate,Sculpture Garden,Harbor / Marina
3,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Pizza Place,Café,Pub,Bakery,Indian Restaurant,Italian Restaurant,Park,Butcher
4,Central Bay Street,Coffee Shop,Italian Restaurant,Bar,Sandwich Place,Café,Ice Cream Shop,Burger Joint,Bubble Tea Shop,Salad Place,Thai Restaurant
5,"Chinatown, Grange Park, Kensington Market",Café,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Coffee Shop,Mexican Restaurant,Chinese Restaurant,Dumpling Restaurant,Grocery Store
6,Christie,Grocery Store,Café,Park,Restaurant,Baby Store,Diner,Nightclub,Italian Restaurant,Convenience Store,Coffee Shop
7,Church and Wellesley,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Gay Bar,Restaurant,Burger Joint,Fast Food Restaurant,Café,Men's Store,Mediterranean Restaurant
8,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Bakery,Deli / Bodega,Gastropub,Seafood Restaurant,Steakhouse
9,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Italian Restaurant,Gastropub,Gym,Deli / Bodega,Bar


## 4. Cluster Neighborhoods

Run k-means to cluster the neighborhoods into 5 clusters.

In [150]:
# set number of clusters
kclusters = 5
downtown_toronto_grouped_clustering = downtown_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 0, 1, 1, 3, 4, 1, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [151]:
downtown_toronto_merged = df_DowntownToronto

# add clustering labels
downtown_toronto_merged['Cluster Labels'] = kmeans.labels_

#merge downtown_toronto_grouped with downtown_toronto_data to add latitude/longiture for each neighborhood
downtown_toronto_merged = downtown_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
downtown_toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Doner Restaurant
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,1,Coffee Shop,Restaurant,Pizza Place,Café,Pub,Bakery,Indian Restaurant,Italian Restaurant,Park,Butcher
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Gay Bar,Restaurant,Burger Joint,Fast Food Restaurant,Café,Men's Store,Mediterranean Restaurant
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,1,Coffee Shop,Café,Bakery,Park,Pub,Restaurant,Mexican Restaurant,Theater,Breakfast Spot,Gym / Fitness Center
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Bubble Tea Shop,Tea Room,Bar,Ramen Restaurant,Sandwich Place


Finally, let's visualize the resulting clusters

In [152]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
print(x)
ys = [i +x+(i*x)**2 for i in range(kclusters)]

print(ys)

[0 1 2 3 4]
[array([0, 1, 2, 3, 4]), array([ 1,  3,  7, 13, 21]), array([ 2,  7, 20, 41, 70]), array([  3,  13,  41,  87, 151]), array([  4,  21,  70, 151, 264])]


In [153]:
colors_array=cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors= []

for lat, lon, poi, cluster in zip(downtown_toronto_merged['Latitude'], downtown_toronto_merged['Longitude'], downtown_toronto_merged['Neighborhood'],
                                 downtown_toronto_merged['Cluster Labels']):
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)
        
map_clusters

5. Examine Clusters
Let's examine each cluster and determine the discriminating venue categories that distinguish each cluster.

#### Cluster 1


In [154]:
cluster1 = downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 0, downtown_toronto_merged.columns[[1] + 
                                                                list(range(5, downtown_toronto_merged.shape[1]))]]

In [155]:
print("Japanese Restaurant Cluster")
cluster1

Japanese Restaurant Cluster


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Gay Bar,Restaurant,Burger Joint,Fast Food Restaurant,Café,Men's Store,Mediterranean Restaurant


In [156]:
cluster2 = downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 1, downtown_toronto_merged.columns[[1] + 
                                                                list(range(5, downtown_toronto_merged.shape[1]))]]

In [157]:
print("Coffee Shop and Cafe Cluster")
cluster2

Coffee Shop and Cafe Cluster


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Park,Playground,Trail,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Doner Restaurant
1,Downtown Toronto,1,Coffee Shop,Restaurant,Pizza Place,Café,Pub,Bakery,Indian Restaurant,Italian Restaurant,Park,Butcher
3,Downtown Toronto,1,Coffee Shop,Café,Bakery,Park,Pub,Restaurant,Mexican Restaurant,Theater,Breakfast Spot,Gym / Fitness Center
4,Downtown Toronto,1,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Bubble Tea Shop,Tea Room,Bar,Ramen Restaurant,Sandwich Place
7,Downtown Toronto,1,Coffee Shop,Italian Restaurant,Bar,Sandwich Place,Café,Ice Cream Shop,Burger Joint,Bubble Tea Shop,Salad Place,Thai Restaurant
8,Downtown Toronto,1,Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Asian Restaurant,Hotel,Bakery,Bar,Clothing Store
9,Downtown Toronto,1,Coffee Shop,Hotel,Aquarium,Café,Pizza Place,Restaurant,Scenic Lookout,Bakery,Italian Restaurant,Brewery
10,Downtown Toronto,1,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Italian Restaurant,Gastropub,Gym,Deli / Bodega,Bar
12,Downtown Toronto,1,Café,Restaurant,Bar,Japanese Restaurant,Bookstore,Coffee Shop,Bakery,Beer Store,Beer Bar,Sandwich Place
13,Downtown Toronto,1,Café,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Coffee Shop,Mexican Restaurant,Chinese Restaurant,Dumpling Restaurant,Grocery Store


In [158]:
cluster3 = downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 2, downtown_toronto_merged.columns[[1] + 
                                                                list(range(5, downtown_toronto_merged.shape[1]))]]

In [159]:
print("Airport Cluster")
cluster3

Airport Cluster


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,2,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Boutique,Airport,Airport Food Court,Airport Gate,Sculpture Garden,Harbor / Marina


In [160]:
cluster4 = downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 3, downtown_toronto_merged.columns[[1] + 
                                                                list(range(5, downtown_toronto_merged.shape[1]))]]

In [161]:
print("Coffer Shop, Cafe, Hotel Cluster")
cluster4

Coffer Shop, Cafe, Hotel Cluster


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Downtown Toronto,3,Coffee Shop,Hotel,Café,Restaurant,Bakery,Italian Restaurant,Cosmetics Shop,Cocktail Bar,Gastropub,Japanese Restaurant
11,Downtown Toronto,3,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Bakery,Deli / Bodega,Gastropub,Seafood Restaurant,Steakhouse


In [162]:
cluster5 = downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 4, downtown_toronto_merged.columns[[1] + 
                                                                list(range(5, downtown_toronto_merged.shape[1]))]]

In [163]:
print("Coffee Shop, Cocktail Bar, Restaurant Cluster")
cluster5


Coffee Shop, Cocktail Bar, Restaurant Cluster


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,4,Coffee Shop,Restaurant,Cocktail Bar,Seafood Restaurant,Farmers Market,Bakery,Italian Restaurant,Steakhouse,Cheese Shop,Café
